In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

# Loading the image from the specified directory address
bird = cv2.imread('/images/bird.jpg')
bird = cv2.cvtColor(bird, cv2.COLOR_BGR2RGB)
gray_bird = cv2.cvtColor(bird, cv2.COLOR_RGB2GRAY)

# Thresholds

In [ ]:
thresholds = [100, 150, 200]

# Set the number of rows and columns for subplots
num_rows = len(thresholds)
num_columns = 2  # Two columns: grayscale and binary images

# Create a figure with subplots
fig, axs = plt.subplots(num_rows, num_columns, figsize=(10, 10))

# Iterate over the thresholds and display the images
for i, threshold in enumerate(thresholds):
    # Perform thresholding by comparing the pixel values with the threshold
    _, binary_image = cv2.threshold(gray_bird, threshold, 255, cv2.THRESH_BINARY)

    # Display the grayscale image
    axs[i, 0].imshow(gray_bird, cmap="gray")
    axs[i, 0].axis('off')

    # Display the thresholded binary image
    axs[i, 1].imshow(binary_image, cmap="gray")
    axs[i, 1].axis('off')
    axs[i, 1].set_title('Threshold = {}'.format(threshold))

# Show the plot
plt.show()

# Region-based Segmentation

In [ ]:
# Set the threshold value
threshold = 100

# Perform thresholding by comparing the pixel values with the threshold
_, binary_image = cv2.threshold(gray_bird, threshold, 255, cv2.THRESH_BINARY)

# Apply connected component analysis to identify regions
num_labels, labels = cv2.connectedComponents(binary_image)

# Create an empty mask to store the segmented region
segmented_image = np.zeros_like(bird)

# Iterate over the labels and assign random colors to each region
for label in range(1, num_labels):
    mask = labels == label
    segmented_image[mask] = np.random.randint(0, 255, size=3)

# Display the segmented image
plt.imshow(segmented_image)
plt.axis('off')
plt.title("Region-Based Segmentation")
plt.show()

# Edge Detection

In [ ]:
# Step Edges - Canny Edge Detection
step_edges = cv2.Canny(gray_bird, 100, 200)

# Ramp Edges - Sobel Operator
sobel_x = cv2.Sobel(gray_bird, cv2.CV_64F, 1, 0, ksize=3)
sobel_y = cv2.Sobel(gray_bird, cv2.CV_64F, 0, 1, ksize=3)
magnitude = np.sqrt(sobel_x ** 2 + sobel_y ** 2)
ramp_edges = np.uint8(magnitude)

# Ridge Edges - Laplacian of Gaussian (LoG)
log_filter = cv2.GaussianBlur(gray_bird, (3, 3), 0)
log_edges = cv2.Laplacian(log_filter, cv2.CV_64F)

# Display the images
plt.figure(figsize=(10, 10))

plt.subplot(1, 3, 1)
plt.imshow(step_edges, cmap='gray')
plt.title("Step Edges")
plt.axis('off')

plt.subplot(1, 3, 2)
plt.imshow(ramp_edges, cmap='gray')
plt.title("Ramp Edges")
plt.axis('off')

plt.subplot(1, 3, 3)
plt.imshow(log_edges, cmap='gray')
plt.title("Ridge Edges")
plt.axis('off')

plt.tight_layout()
plt.show()

# Clustering

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin_min, completeness_score, homogeneity_score

# Reshape the grayscale image into a 2D array
gray_bird_2d = gray_bird.reshape(-1, 1)

# Define a list of k values to loop over
k_values = [2, 3, 4, 5]

best_purity = -1
best_k = -1

# Loop over different k values
for k in k_values:
    # Perform k-means clustering
    kmeans = KMeans(n_clusters=k, random_state=0)
    kmeans.fit(gray_bird_2d)

    # Get the labels and centroids of the clusters
    labels = kmeans.labels_
    centroids = kmeans.cluster_centers_

    # Reshape the labels back into the shape of the image
    segmented_image = labels.reshape(gray_bird.shape)

    # Calculate evaluation metrics
    compactness_avg = np.mean(pairwise_distances_argmin_min(gray_bird_2d, centroids)[1])
    purity_completeness = completeness_score(labels, pairwise_distances_argmin_min(gray_bird_2d, centroids)[0])
    purity_homogeneity = homogeneity_score(labels, pairwise_distances_argmin_min(gray_bird_2d, centroids)[0])

    if purity_completeness + purity_homogeneity > best_purity:
        best_purity = purity_completeness + purity_homogeneity
        best_k = k

# Print the best k-value
print("Best K:", best_k)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [ ]:
# Perform k-means clustering with k=2
kmeans = KMeans(n_clusters=2, random_state=0)
kmeans.fit(gray_bird_2d)
labels = kmeans.labels_
segmented_image = labels.reshape(gray_bird.shape)

# Display the segmented image
plt.imshow(segmented_image)
plt.title("K-Means Segmentation (k=2)")
plt.axis('off')
plt.show()

# Watershed segmentation

In [ ]:
# Apply adaptive thresholding to obtain a binary image
_, binary_image = cv2.threshold(gray_bird, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

# Perform morphological operations to remove noise and enhance the segmented regions
kernel = np.ones((3, 3), np.uint8)
opening = cv2.morphologyEx(binary_image, cv2.MORPH_OPEN, kernel, iterations=2)
sure_bg = cv2.dilate(opening, kernel, iterations=3)
sure_bg = np.uint8(sure_bg)

# Apply distance transform to find the areas sure to be foreground
dist_transform = cv2.distanceTransform(opening, cv2.DIST_L2, 5)
_, sure_fg = cv2.threshold(dist_transform, 0.7*dist_transform.max(), 255, 0)
sure_fg = np.uint8(sure_fg)

# Identify the areas of uncertainty by subtracting the foreground from the background
unknown = cv2.subtract(sure_bg, sure_fg)

# Mark the regions of foreground and uncertainty with labels
_, markers = cv2.connectedComponents(sure_fg)
markers += 1
markers[unknown == 255] = 0

# Apply watershed segmentation to segment the regions
segmented_image = cv2.watershed(bird, markers)

# Color the segmented regions for visualization
bird[segmented_image == -1] = [0, 0, 255]

# Display the segmented image
plt.imshow(cv2.cvtColor(bird, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.title("Watershed Segmentation")
plt.show()